Cleaning Corpus Text: Endangered Species Act

In [1]:
# Load libraries
library(readr)
library(dplyr)
library(tidytext)
library(tm)
library(textmineR)
library(SnowballC)
library(textstem)
library(textclean)
library(stringr)
library(qdapDictionaries)
library(lexicon)
library(here)
library(text2vec)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: NLP

Loading required package: Matrix


Attaching package: ‘textmineR’


The following object is masked from ‘package:Matrix’:

    update


The following object is masked from ‘package:stats’:

    update


Loading required package: koRpus.lang.en

Loading required package: koRpus

Loading required package: sylly

For information on available language packages for 'koRpus', run

  available.koRpus.lang()

and see ?install.koRpus.lang()



Attaching package: ‘koRpus’


The following object is masked from ‘package:tm’:

    readTagged


The following object is masked from ‘package:readr’:

    tokenize


here() starts at /home/ec2-user/SageMaker/esa_analysis



In [2]:
i_am("here.txt")
here()

here() starts at /home/ec2-user/SageMaker/esa_analysis



[1] "/home/ec2-user/SageMaker/esa_analysis"

In [4]:
# Read in the original CSV file (replace blank.csv with csv name)
original_corpus <- read.csv(
    here("output_file", "esa_10_22_2025.csv"))

str(original_corpus)

'data.frame':	5241 obs. of  5 variables:
 $ X    : int  0 1 2 3 4 5 6 7 8 9 ...
 $ GOID : num  4.22e+08 4.09e+08 4.29e+08 4.19e+08 2.07e+09 ...
 $ Title: chr  "Developers Urge Delay of Gnatcatcher Ruling; Birds: Species isn't in danger, they claim. Federal decision this "| __truncated__ "TODAY IN CONGRESS" "Plan Offered to Aid Northwest Salmon and Trout" "SAVING WILDLIFE" ...
 $ Date : chr  "2000-09-26" "1999-07-20" "1994-03-27" "1999-07-05" ...
 $ Text : chr  "Armed with a new study, developers are demanding that federal officials postpone a decision due later this week"| __truncated__ "SENATE Meets at 9:30 a.m. Committees: Armed Services -- 9:30 a.m. U.S. policy & military operations regarding K"| __truncated__ "AP The Clinton Administration has proposed protection zones along rivers and streams to help threatened fish sp"| __truncated__ "In the Tribune's June 18 news item on the recovery and proposed removal of the bald eagle from the endangered s"| __truncated__ ...


In [5]:
# Remove rows where Text is empty or contains only whitespace using dplyr
original_corpus <- original_corpus %>%
  filter(nchar(trimws(Text)) > 0)

str(original_corpus)

'data.frame':	5239 obs. of  5 variables:
 $ X    : int  0 1 2 3 4 5 6 7 8 9 ...
 $ GOID : num  4.22e+08 4.09e+08 4.29e+08 4.19e+08 2.07e+09 ...
 $ Title: chr  "Developers Urge Delay of Gnatcatcher Ruling; Birds: Species isn't in danger, they claim. Federal decision this "| __truncated__ "TODAY IN CONGRESS" "Plan Offered to Aid Northwest Salmon and Trout" "SAVING WILDLIFE" ...
 $ Date : chr  "2000-09-26" "1999-07-20" "1994-03-27" "1999-07-05" ...
 $ Text : chr  "Armed with a new study, developers are demanding that federal officials postpone a decision due later this week"| __truncated__ "SENATE Meets at 9:30 a.m. Committees: Armed Services -- 9:30 a.m. U.S. policy & military operations regarding K"| __truncated__ "AP The Clinton Administration has proposed protection zones along rivers and streams to help threatened fish sp"| __truncated__ "In the Tribune's June 18 news item on the recovery and proposed removal of the bald eagle from the endangered s"| __truncated__ ...


In [6]:
# Save the corpus as a csv file
write.csv(
    original_corpus,
    here("data", "original", "original_corpus_filtered_empty_docs.csv")
)

In [7]:
# Function for preserving abbreviations
preserve_abbreviations <- function(text) {
  matches <- gregexpr("\\b([a-z](?:\\.[a-z])+)\\.?", text, ignore.case = TRUE)
  regmatches(text, matches) <- lapply(regmatches(text, matches), function(abbrev) {
    gsub("\\.", "DOTDOTDOT", abbrev)
  })

  return(text)
}

In [8]:
# Function to convert possessive forms
convert_possessives <- function(text) {
  text <- gsub("\\b(\\w+)'s\\b", "\\1 ", text, ignore.case = TRUE)
  text <- gsub("\\b(\\w+)'\\b", "\\1 ", text, ignore.case = TRUE)

  return(text)
}

In [9]:
corpus_clean <- original_corpus %>% 
  mutate(
      Text = as.character(Text),
      Text = preserve_abbreviations(Text),
      Text = gsub("\\b\\S*\\.(com|org|gov|edu|htm|net)\\S*\\b", " ", Text, ignore.case = TRUE),
      Text = gsub("\\S*@\\S*", " ", Text, ignore.case = TRUE),
      Text = gsub("\\S*\\d+\\S*", " ", Text, ignore.case = TRUE),
      Text = replace_contraction(Text),
      Text = gsub(paste0("\\b(", paste(stopwords("en"), collapse = "|"), ")\\b"), " ", Text, ignore.case = TRUE),
      Text = gsub(paste0("\\b(", paste(BuckleySaltonSWL, collapse = "|"), ")\\b"), " ", Text, ignore.case = TRUE),
      Text = convert_possessives(Text),
      Text = replace_ordinal(Text, num.paste = TRUE, remove = TRUE),
      Text = replace_number(Text, num.paste = TRUE, remove = TRUE),
      Text = add_comma_space(Text)
  )

In [10]:
lemma_dict <- make_lemma_dictionary(
    corpus_clean$Text,
    engine = "lexicon"
)

final_lemma_dict <- lemma_dict %>% 
    filter(nchar(trimws(token)) > 1)

str(final_lemma_dict)
final_lemma_dict

Warning message:
“`tbl_df()` was deprecated in dplyr 1.0.0.
ℹ Please use `tibble::as_tibble()` instead.
ℹ The deprecated feature was likely used in the textstem package.
  Please report the issue to the authors.”


'data.frame':	15907 obs. of  2 variables:
 $ token: chr  "abalones" "abandoned" "abandoning" "abandons" ...
 $ lemma: chr  "abalone" "abandon" "abandon" "abandon" ...
 - attr(*, ".internal.selfref")=<externalptr> 
 - attr(*, "sorted")= chr "token"


token,lemma
<chr>,<chr>
abalones,abalone
abandoned,abandon
abandoning,abandon
abandons,abandon
abashed,abash
abated,abate
abating,abate
abbreviated,abbreviate
abdicated,abdicate


In [11]:
corpus_more_clean <- corpus_clean  %>% 
  mutate( 
      Text = lemmatize_strings(Text, final_lemma_dict),
      Text = strip(Text, char.keep = NULL, apostrophe.remove = TRUE),
      Text = gsub("DOTDOTDOT", ".", Text, ignore.case = TRUE),
      Text = stripWhitespace(Text)
  )

In [12]:
str(original_corpus)
head(original_corpus)

'data.frame':	5239 obs. of  5 variables:
 $ X    : int  0 1 2 3 4 5 6 7 8 9 ...
 $ GOID : num  4.22e+08 4.09e+08 4.29e+08 4.19e+08 2.07e+09 ...
 $ Title: chr  "Developers Urge Delay of Gnatcatcher Ruling; Birds: Species isn't in danger, they claim. Federal decision this "| __truncated__ "TODAY IN CONGRESS" "Plan Offered to Aid Northwest Salmon and Trout" "SAVING WILDLIFE" ...
 $ Date : chr  "2000-09-26" "1999-07-20" "1994-03-27" "1999-07-05" ...
 $ Text : chr  "Armed with a new study, developers are demanding that federal officials postpone a decision due later this week"| __truncated__ "SENATE Meets at 9:30 a.m. Committees: Armed Services -- 9:30 a.m. U.S. policy & military operations regarding K"| __truncated__ "AP The Clinton Administration has proposed protection zones along rivers and streams to help threatened fish sp"| __truncated__ "In the Tribune's June 18 news item on the recovery and proposed removal of the bald eagle from the endangered s"| __truncated__ ...


X GOID      
1 0  421721236
2 1  408507519
3 2  429461998
4 3  418798522
5 4 2073930752
6 5  419462981
  Title                                                                                                                                                  
1 Developers Urge Delay of Gnatcatcher Ruling; Birds: Species isn't in danger, they claim. Federal decision this week could protect almost 800,000 acres.
2 TODAY IN CONGRESS                                                                                                                                      
3 Plan Offered to Aid Northwest Salmon and Trout                                                                                                         
4 SAVING WILDLIFE                                                                                                                                        
5 Wildlife Threatened Again                                                                                                                              
6 County puts out rabid bat warning                                                                                                                      
  Date      
1 2000-09-26
2 1999-07-20
3 1994-03-27
4 1999-07-05
5 2018-07-24
6 2001-07-24
  Text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [14]:
str(corpus_clean)
head(corpus_clean)

'data.frame':	5239 obs. of  5 variables:
 $ X    : int  0 1 2 3 4 5 6 7 8 9 ...
 $ GOID : num  4.22e+08 4.09e+08 4.29e+08 4.19e+08 2.07e+09 ...
 $ Title: chr  "Developers Urge Delay of Gnatcatcher Ruling; Birds: Species isn't in danger, they claim. Federal decision this "| __truncated__ "TODAY IN CONGRESS" "Plan Offered to Aid Northwest Salmon and Trout" "SAVING WILDLIFE" ...
 $ Date : chr  "2000-09-26" "1999-07-20" "1994-03-27" "1999-07-05" ...
 $ Text : chr  "Armed       study, developers   demanding   federal officials postpone   decision due     week   providing     "| __truncated__ "SENATE Meets     aDOTDOTDOTmDOTDOTDOT Committees: Armed Services --   aDOTDOTDOTmDOTDOTDOT UDOTDOTDOTSDOTDOTDOT"| __truncated__ "AP   Clinton Administration   proposed protection zones   rivers   streams     threatened fish species   Federa"| __truncated__ "Tribune  June   news item     recovery   proposed removal     bald eagle     endangered species list,   mention"| __truncated__ ...


X GOID      
1 0  421721236
2 1  408507519
3 2  429461998
4 3  418798522
5 4 2073930752
6 5  419462981
  Title                                                                                                                                                  
1 Developers Urge Delay of Gnatcatcher Ruling; Birds: Species isn't in danger, they claim. Federal decision this week could protect almost 800,000 acres.
2 TODAY IN CONGRESS                                                                                                                                      
3 Plan Offered to Aid Northwest Salmon and Trout                                                                                                         
4 SAVING WILDLIFE                                                                                                                                        
5 Wildlife Threatened Again                                                                                                                              
6 County puts out rabid bat warning                                                                                                                      
  Date      
1 2000-09-26
2 1999-07-20
3 1994-03-27
4 1999-07-05
5 2018-07-24
6 2001-07-24
  Text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [13]:
str(corpus_more_clean)
head(corpus_more_clean)

'data.frame':	5239 obs. of  5 variables:
 $ X    : int  0 1 2 3 4 5 6 7 8 9 ...
 $ GOID : num  4.22e+08 4.09e+08 4.29e+08 4.19e+08 2.07e+09 ...
 $ Title: chr  "Developers Urge Delay of Gnatcatcher Ruling; Birds: Species isn't in danger, they claim. Federal decision this "| __truncated__ "TODAY IN CONGRESS" "Plan Offered to Aid Northwest Salmon and Trout" "SAVING WILDLIFE" ...
 $ Date : chr  "2000-09-26" "1999-07-20" "1994-03-27" "1999-07-05" ...
 $ Text : chr  "arm study developer demand federal official postpone decision due week provide acre critical habitat tiny gnatc"| __truncated__ "senate meet a.m. committee arm service a.m. u.s. policy military operation kosovo defense sec william cohen joi"| __truncated__ "ap clinton administration propose protection zone river stream threaten fish species federal land eastern orego"| __truncated__ "tribune june news item recovery propose removal bald eagle endanger species list mention eagle extinction misle"| __truncated__ ...


,X,GOID,Title,Date,Text
,<int>,<dbl>,<chr>,<chr>,<chr>
1,0,421721236,"Developers Urge Delay of Gnatcatcher Ruling; Birds: Species isn't in danger, they claim. Federal decision this week could protect almost 800,000 acres.",2000-09-26,arm study developer demand federal official postpone decision due week provide acre critical habitat tiny gnatcatcher southern california study write jonathon l atwood biologist early research conclude bird near extinction developer proof california gnatcatcher genetically close mexican songbird species danger environmentalist scientist disagree california bird mexican thousand pair gnatcatchers california million mexico study fund build industry assn southern california transportation corridor agency u.s. navy write robert m zink george f barrowclough rachelle c blackwell rago atwood study compare dna gnatcatchers mexico dna obtain feather nestling unite state put simply base dna datum northern population constitute unique component gnatcatcher biodiversity study conclude scientist caution test small numb bird evidence genetic mutation cross breed mexican american bird letter mail monday irvine attorney rob thornton urge interior secretary bruce babbitt postpone decision critical habitat bird judge order federal official designate critical habitat bird saturday u.s. fish wildlife official seek emergency extension court bypass order atwood report california gnatcatchers distinct mexican cousin primary scientific evidence government rely decide list gnatcatcher threaten species thornton represent transportation corridor agency pulte home corp forest lawn memorial park assn impossible determine gnatcatcher acre essential fact threaten environmental group genetic similarity mexican bird obvious difference birds color final habitat decision vital gnatcatcher sage scrub nest siege development endanger species act narrow inflexible genetic definition species andrew wetzler attorney natural resource defense council environmentalist note healthy population neighbor country species safe unite state kimball garrett ornithology collection manager natural history museum los angeles county cite birds appearance proof difference california bird dark mexican bird white breast tail caption photo male california gnatcatcher photographer al schaben los angeles time photo slope eastern transportation corridor toll road orange coastal sage scrub habitat gnatcatcher photographer don kelsen los angeles time credit time staff writer
2,1,408507519,TODAY IN CONGRESS,1999-07-20,senate meet a.m. committee arm service a.m. u.s. policy military operation kosovo defense sec william cohen joint chief chairman gen henry shelton hart office bldg arm service a.m. nomination f whitten peter secretary air force arthur money assistant secretary defense command control communication intelligence russell office bldg budget a.m. mid session review president fy budget dirksen office bldg energy natural resource p.m. forest public land management subc national monument public participation act dob environment public work a.m. fishery wildlife drink water subc habitat conservation plan endanger species act dob foreign relation a.m. nomination ambassador philippines indonesia palau fiji nauru tonga tuvalu brunei darussalam dob foreign relation p.m. international operation subc close u.n. international criminal court dob governmental affair a.m. permanent investigation subc deceptive mail practice include sweepstake skill contest alike mailing legislation control practice dob health education labor pension a.m. elementary secondary education act reauth dob special age p.m. impact drug switch old american dob house meet a.m. committee agriculture a.m. general farm commodity subc small watershed rehabilitation amendment longworth house office bldg appropriation a.m. mark d.c. energy water foreign operation approps fy rayburn house office bldg bank financial service a.m. financial institution consumer credit subc financial privacy issue rhob co

In [15]:
# Save the corpus as a csv file
write.csv(
    corpus_more_clean,
    here("data", "processed", "clean_text_corpus.csv")
)

In [ ]:
# Testing
#Text <- c(
#    "TestIng/gr.com/ tESt/.Net/org",
#    ".Net/org Test .com can't   S. Lily's",
#    "   S. Lily's first PlAce 1st, tests patches",
#    "howtotestthis.com/",
#    "U.S.A. u.s.a lily356@edu.net/",
#    "lily356@edu.net/ gr8t. --g-5_ GrizZly bear.!",
#    "howtotestthis.net hottohelp.com grizzlies-",
#    "testhelp.com/help.org testhelp.com/help.org/",
#    ",GRIZZLY BEAR:  GRIZZLY-BEAR, grizzlies testing",
#    " midflorida.com.GRIZZLY BEAR: ",
#    "grizzlybearsarecool coolgrizzliesbear"
#)
#GOID <- c(11345, 324, 876, 335, 987, 574, 9, 87, 768, 24, 5)

#sample_corpus <- data.frame(GOID = GOID, Text = Text)
#sample_corpus